In [ ]:
# --- CELDA 1: INSTALACIÓN DE DEPENDENCIAS ---
# Ejecuta esta celda solo una vez al inicio de tu sesión de trabajo.

print("--- [CELDA 1] Instalando/actualizando 'ultralytics' ---")
!pip install -U ultralytics --quiet
print("✅ Instalación completada. Por favor, reinicia el entorno de ejecución ahora.")

--- [CELDA 1] Instalando/actualizando 'ultralytics' ---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.6 MB/s eta 0:00:00
✅ Instalación completada. Por favor, reinicia el entorno de ejecuci

In [ ]:
# --- CELDA 2: CONFIGURACIÓN Y VERIFICACIÓN ---
# Ejecuta esta celda DESPUÉS de haber reiniciado el entorno.

import os
import shutil
from pathlib import Path
import torch

print("--- [CELDA 2] CONFIGURANDO EL ENTORNO DE ENTRENAMIENTO ---")

# 1. Montaje de Drive
print("--- Montando Google Drive ---")
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 2. VERIFICACIÓN DE GPU
print("\n--- Verificando la GPU disponible ---")
!nvidia-smi
if not torch.cuda.is_available():
    print("\n❌ ¡ATENCIÓN! No se detectó una GPU.")
else:
    print("\n✅ GPU detectada. ¡Listos para entrenar!")

# 3. PARÁMETROS Y RUTAS
print("\n--- Definiendo parámetros y rutas ---")
DATASET_YAML_PATH = "/content/drive/MyDrive/Coffee_Dataset_Balanced_v3/data.yaml"
MODELS_DESTINATION_DIR = Path('/content/drive/MyDrive/YOLOv8_Trained_Models/')
MODEL_VARIANT = 'yolov8x-seg.pt'
PROJECT_NAME = 'Coffee_Seg_Final_Run'
IMG_SIZE = 1024
BATCH_SIZE = 16
EPOCHS = 200
PATIENCE = 40
OPTIMIZER = 'AdamW'
RUN_NAME = f'run_{MODEL_VARIANT.split(".")[0]}_img{IMG_SIZE}_b{BATCH_SIZE}_balanced'

print("Configuración cargada:")
print(f"  - Dataset: {DATASET_YAML_PATH}")
print(f"  - Destino de Modelos: {MODELS_DESTINATION_DIR}")
print(f"  - Ejecución (Run): {RUN_NAME}")

# 4. Crear carpeta de destino para el modelo
os.makedirs(MODELS_DESTINATION_DIR, exist_ok=True)
print(f"\n✅ Carpeta de destino para modelos asegurada en Drive.")

print("\n--- FIN DE CELDA 2 ---")

--- [CELDA 2] CONFIGURANDO EL ENTORNO DE ENTRENAMIENTO ---
--- Montando Google Drive ---
Mounted at /content/drive

--- Verificando la GPU disponible ---
Fri Jun 20 16:58:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             45W /  400W |       0MiB /  40960MiB |      0%  

In [ ]:
# --- CELDA 3: ENTRENAMIENTO CON GUARDADO AUTOMÁTICO DE RESPALDO ---
from ultralytics import YOLO
import traceback
import shutil
from pathlib import Path

# Las variables se definieron en la Celda 2
print(f"--- [CELDA 3] PREPARANDO ENTRENAMIENTO PARA EL RUN: {RUN_NAME} ---")

model = YOLO(MODEL_VARIANT)

try:
    print(f"\n>>> Iniciando entrenamiento: {RUN_NAME} <<<")
    model.train(
        data=DATASET_YAML_PATH,
        project=PROJECT_NAME,
        name=RUN_NAME,
        epochs=EPOCHS,
        patience=PATIENCE,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        optimizer=OPTIMIZER,
        mosaic=1.0,
        copy_paste=0.5,
        mixup=0.1,
    )
    print(f"\n--- ✅ ENTRENAMIENTO COMPLETADO NORMALMENTE ---")

except KeyboardInterrupt:
    print("\n--- 🛑 ENTRENAMIENTO INTERRUMPIDO POR EL USUARIO ---")

except Exception as e:
    print(f"\n❌ Ocurrió un error catastrófico durante el entrenamiento:")
    traceback.print_exc()

finally:
    print("\n--- INICIANDO PROTOCOLO DE GUARDADO DE RESPALDO ---")
    source_model_path = Path(f'./{PROJECT_NAME}/{RUN_NAME}/weights/best.pt')
    destination_model_path = MODELS_DESTINATION_DIR / f'best_model_{RUN_NAME}.pt'
    if source_model_path.exists():
        print(f"✅ Se encontró 'best.pt'. Copiando a tu Google Drive...")
        shutil.copy2(source_model_path, destination_model_path)
        if destination_model_path.exists():
            print(f"✅ ¡Éxito! El mejor modelo ha sido guardado en tu Drive.")
        else:
            print(f"❌ ERROR: La copia falló.")
    else:
        print("🟡 AVISO: No se encontró 'best.pt'. (Normal si el entrenamiento se detuvo antes de la primera validación).")

    print("\n--- FIN DE CELDA 3 ---")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
--- [CELDA 3] PREPARANDO ENTRENAMIENTO PARA EL RUN: run_yolov8x-seg_img1024_b16_balanced ---


100%|██████████| 137M/137M [00:00<00:00, 240MB/s]



>>> Iniciando entrenamiento: run_yolov8x-seg_img1024_b16_balanced <<<
Ultralytics 8.3.157 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.5, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Coffee_Dataset_Balanced_v3/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8x-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=run_yolov8x-seg_img1024_b16_bala

100%|██████████| 755k/755k [00:00<00:00, 14.1MB/s]

Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              


...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  15%|█▌        | 2/13 [00:01<00:07,  1.50it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:09<00:00,  1.39it/s]


                   all        392       5791      0.767      0.818      0.848      0.809      0.768      0.819      0.849      0.804
              Inmaduro        345       4236       0.91      0.941       0.97       0.92       0.91      0.941       0.97      0.911
                Pinton         96        446      0.752      0.713      0.794      0.761      0.752      0.713      0.793      0.755
                Maduro        115        800      0.795      0.825      0.871      0.834      0.795      0.825      0.872      0.818
           Sobremaduro         45        122      0.613      0.648      0.694      0.658      0.613      0.648      0.694      0.668
                  Seco         57        187      0.766      0.963      0.912      0.874       0.77      0.968      0.915      0.865
Speed: 0.3ms preprocess, 10.4ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to Coffee_Seg_Final_Run/run_yolov8x-seg_img1024_b16_balanced

--- ✅ ENTRENAMIENTO COMPLETADO NORMALMENTE 